In [1]:
import pandas as pd
import numpy as np
import yfinance as yf # requires yfinance - pip install yfinance

In [2]:
tick = yf.Ticker('SPY') # S&P500
data = tick.history(start="2019-01-01", end="2019-11-19", rounding=True)

In [3]:
data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2018-12-31,246.13,246.75,244.07,246.48,144299400,0.0,0
2019-01-02,242.60,247.75,242.57,246.74,126925200,0.0,0
2019-01-03,244.81,245.15,240.32,240.85,144140700,0.0,0
2019-01-04,244.18,249.63,243.77,248.92,142628800,0.0,0
2019-01-07,249.21,252.43,248.23,250.88,103139100,0.0,0


In [4]:
h=data[-224:].Close
window=125
errpct = 0.005
hough_scale=0.01
hough_prob_iter=10 
sortError=False

In [5]:
min_h, max_h, len_h = min(h), max(h), len(h)
scale = (max_h - min_h) / len_h
fltpct = scale * errpct

In [13]:
minimaIdxs = np.flatnonzero(h.rolling(window=3, min_periods=1, center=True).aggregate(lambda x: len(x) == 3 and x[0] > x[1] and x[2] > x[1])).tolist()
maximaIdxs = np.flatnonzero(h.rolling(window=3, min_periods=1, center=True).aggregate(lambda x: len(x) == 3 and x[0] < x[1] and x[2] < x[1])).tolist()


In [7]:
hs = h.loc[h.shift(-1) != h]
x = hs.rolling(window=3, center=True).aggregate(lambda x: x[0] > x[1] and x[2] > x[1])
minimaIdxs2 = [h.index.get_loc(y) for y in x[x == 1].index]
x = hs.rolling(window=3, center=True).aggregate(lambda x: x[0] < x[1] and x[2] < x[1])
maximaIdxs2 = [h.index.get_loc(y) for y in x[x == 1].index]


In [15]:
minimaIdxs==minimaIdxs2

True

In [16]:
maximaIdxs==maximaIdxs2

True

In [14]:
from findiff import FinDiff

dx = 1 #1 day interval
d_dx = FinDiff(0, dx, 1)
d2_dx2 = FinDiff(0, dx, 2)
clarr = np.asarray(h)
mom = d_dx(clarr)
momacc = d2_dx2(clarr)
        
def get_extrema(isMin): #numerical derivative will yield prominent extrema points only
    return [x for x in range(len(mom))
            if (momacc[x] > 0 if isMin else momacc[x] < 0) and
              (mom[x] == 0 or #either slope is 0, or it crosses from positive to negative with the closer to 0 of the two chosen or prior if a tie
              (x != len(mom) - 1 and (mom[x] > 0 and mom[x+1] < 0 and h[x] >= h[x+1] or #mom[x] >= -mom[x+1]
                  mom[x] < 0 and mom[x+1] > 0 and h[x] <= h[x+1]) or #-mom[x] >= mom[x+1]) or
                  x != 0 and (mom[x-1] > 0 and mom[x] < 0 and h[x-1] < h[x] or #mom[x-1] < -mom[x] or
                  mom[x-1] < 0 and mom[x] > 0 and h[x-1] > h[x])))] #-mom[x-1] < mom[x])))]

minimaIdxs3, maximaIdxs3 = get_extrema(True), get_extrema(False)

In [17]:
minimaIdxs==minimaIdxs3

False

In [18]:
maximaIdxs==maximaIdxs3

False